In [1]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

This requires having `bnpm`. Get it with `pip install bnpm` or here: https://github.com/RichieHakim/basic_neural_processing_modules

In [3]:
from bnpm import server

In [6]:
import os
from pathlib import Path
import functools
import tempfile
import getpass
import gc

------------
# ==== Start ====

## Prepare username + password

In [8]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')

use_localSshKey = True

pw = server.pw_encode(getpass.getpass(prompt='Password: ')) if use_localSshKey==False else None

path_sshKey = '/home/rich/.ssh/id_rsa' if use_localSshKey else None

Username: rh183


## Prepare `sftp` and `ssh` objects

In [43]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [10]:
## initialize ssh_compute
ssh_c = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [44]:
sftp = server.sftp_interface(ssh_client=ssh_t.client)

In [78]:
prompt_snip = f'[{username}'

In [81]:
send_expect = functools.partial(ssh_c.send_expect, str_success=prompt_snip)

## Start a small interactive node

In [167]:
send_expect(f'srun --pty -p interactive -t 0-00:30:00 --mem=4G /bin/bash');

srun: Force Terminated job 11970866
slurmstepd: error: *** STEP 11970866.0 ON compute-e-16-233 CANCELLED AT 2023-07-02T12:09:26 DUE TO TIME LIMIT ***
srun: Job step aborted: Waiting up to 62 seconds for job step to finish.
srun: error: compute-e-16-233: task 0: Terminated
]$ 


## Pull/update repo

In [172]:
send_expect('cd /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm')
send_expect('git pull');

o/sabatini/rich/virtual_envs/FR
ce-rhythm]$ 
cd /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm
ce-rhythm]$ 


## Activate environment

In [168]:
name_env = '/n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'

send_expect(f'conda activate {name_env}');

srun --pty -p interactive -t 0-00:30:00 --mem=4G /bin/bash
srun: job 11972660 queued and waiting for resources
srun: job 11972660 has been allocated resources

conda activate /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

(base) [rh183@compute-a-16-163 face-rhythm]$ conda a


## Prepare directories

In [196]:
import numpy as np

mouse = 'mouse_0322N'
date = '20230430'

name_slurm = mouse + '_' + date[-4:]

dir_videos      = str(Path('/n/data1/hms/neurobio/sabatini/gyu/data/full_data') / mouse / date / 'camera_data' / 'exp' / 'cam4')

dir_ROIs        = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20230701' / 'ROIs_prealigned' / f'{mouse}__{date}' / 'ROIs.h5')

dir_saveOutputs = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20230701' / date)


name_job = 'jobNum_'


path_script_remote      = str(Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/pipeline_basic.py'))  ## path to a copy of the script (.py) file to run within the repo on the server

path_dispatcher_local   = str(Path(r'/media/rich/Home_Linux_partition/github_repos/face-rhythm/batch_run/').resolve() / 'dispatcher.py')  ## path to the dispatcher.py file on local computer
path_dispatcher_remote  = (Path(dir_saveOutputs) / 'dispatcher.py').as_posix()  ## path to where you want the dispatcher.py file to be copied onto the server

## Prepare commands

In [197]:
## Expectation for dispatch args: 
# path_script, dir_save, dir_videos, dir_ROIs, name_job, name_slurm, name_env


commands = {
#     'make_dir': f"mkdir -p {dir_data_remote}",
#     'copy': f"cp -r {dir_s2p_MICROSCOPE} {dir_data_remote}",
#     'dispatch': f"python {str(path_dispatcher_remote)} {dir_saveOutputs} {path_script_remote} {name_job} {name_slurm} {dir_videos}",  ## dispatcher expecting these args as inputs
    'dispatch': f"\
python3 \
{path_dispatcher_remote} \
{path_script_remote} \
{dir_saveOutputs} \
{dir_videos} \
{dir_ROIs} \
{name_job} \
{name_slurm} \
{name_env}",
}
display(commands)

{'dispatch': 'python3 /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/dispatcher.py /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/pipeline_basic.py /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430 /n/data1/hms/neurobio/sabatini/gyu/data/full_data/mouse_0322N/20230430/camera_data/exp/cam4 /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/ROIs_prealigned/mouse_0322N__20230430/ROIs.h5 jobNum_ mouse_0322N_0430 /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

## Upload `dispatcher.py` file

In [198]:
sftp.mkdir_p(str(Path(path_dispatcher_remote).parent))
sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));

## Dispatch

In [182]:
send_expect(commands['dispatch']);

urobio/sabatini/rich/virtual_envs/FR
dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR
Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/pipeline_basic.py to /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/pipeline_basic.py
Submitting job: jobNum_ 0
sbatch: Setting QOS to gpuquad_qos
Submitted batch job 11973022
ce-rhythm]$ 


## Run with serverteeny

In [207]:
run_file = {
    "name": "Rich",
    "o2_acct": "km335",
    "notes": "testing serverteeny for FR",
    "command": f"sbatch /n/data1/hms/neurobio/sabatini/rich/serverteeny_submitter.sh {commands['dispatch']}",
}

In [204]:
run_file = {
    "name": "Rich",
    "o2_acct": "km335",
    "notes": "testing serverteeny for FR",
    "command": f"sbatch /n/data1/hms/neurobio/sabatini/rich/serverteeny_submitter.sh pip list",
}

In [208]:
path_temp = str(Path(tempfile.gettempdir()) / 'run.json')
bnpm.file_helpers.json_save(
    obj=run_file,
    filepath=path_temp,
)

In [209]:
sftp.sftp.put(
    localpath=path_temp,
    remotepath=str(Path('/n/data1/hms/neurobio/sabatini/serverteeny/run') / Path(path_temp).name),
);

### check on job

In [166]:
ssh_c.send_expect(f'squeue -u {username} --Format=jobid,name,partition,state,timeused,starttime,timelimit,reasonlist,minmemory,mincpus,gres');

sque
me,timelimit,reasonlist,minmemory,mincpus,grese,timeused,startti 
JOBID               NAME                PARTITION           STATE               TIME                START_TIME          TIME_LIMIT          NODELIST(REASON)    MIN_MEMORY          MIN_CPUS            TRES_PER_NODE       
11971088            mouse_0322N_0430    gpu                 PENDING             0:00                N/A                 2:30:00             (None)              36G                 20                  gres:gpu:1,gres:vram
11968637            mouse_0322N_0430    gpu_requeue         PENDING             0:00                2023-07-02T14:12:32 2:30:00             (Resources)         36G                 20                  gres:gpu:1,gres:vram
11970866            bash                interactive         RUNNING             4:34                2023-07-02T11:39:10 30:00               compute-e-16-233    4G                  1                   N/A                 
ce-rhythm]$ s/neurobio/sabatini/rich/virtual_

## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()